In [1]:
import math
import sklearn
import numpy

import theano
import theano.tensor as T
import lasagne

import json

import pandas as pd
from sklearn.cross_validation import train_test_split

import lasagne.regularization as reg

/home/flerchy/.virtualenvs/ml/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
good = open("../scripts/codestyle_stats/fine_res.json", "r")
bad = open("../scripts/codestyle_stats/shit_res.json", "r")

In [3]:
bad_data = bad.read().replace('\n', ' ')
bad_dict = json.loads(bad_data)

In [4]:
good_data = good.read().replace('\n', ' ')
good_dict = json.loads(good_data)

In [5]:
print len(bad_dict)

608


In [6]:
print len(good_dict)
print [0]*75

2540
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
bad = []
for i in range(0, len(bad_dict)):
    s = []
    if ('errors' in bad_dict[i]):
        #print bad_dict[i]['errors']
        s.extend(bad_dict[i]['errors'].values())
        #print s
    else:
        s.extend([0]*(len(bad_dict[0]['errors'])))
    #print s
    #print s
    for key, value in bad_dict[i].items():
         if key not in ('errors', 'filename'):
            s.append(int(value))
    #print data
    s =numpy.array(s) 
    bad.append(s)
bad = numpy.array(bad)
print bad.shape
# for i in range(0, len(X_train)):
#     for elem in X_train[i]:
#         if not (isinstance( elem, int )):
#             print i
#             print type(elem)
#print X_train

(608, 82)


In [8]:
print bad.shape

(608, 82)


In [9]:
good = []
for i in range(0, len(bad_dict)):
    s = []
    if ('errors' in good_dict[i]):
        #print bad_dict[i]['errors']
        s.extend(numpy.array(good_dict[i]['errors'].values()))
        #print s
    else:
        s.extend([0]*(len(bad_dict[0]['errors'])))
    #print s
    #print s
    for key, value in good_dict[i].items():
         if key not in ('errors', 'filename'):
            s.append(int(value))
            #print data
    s = numpy.array(s)
    good.append(s)
good = numpy.array(good)
print good.shape
print good[0]
    #print s

(608, 82)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  1  0 17  0  0]


In [10]:
X_train = []
X_train.extend(good)
X_train.extend(bad)
X_train = numpy.array(X_train)

X_train.shape

(1216, 82)

In [11]:
y_train = numpy.array([1]*len(bad_dict) + [0]* len(bad_dict), dtype=numpy.int32)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4)
for s in (X_train, X_val, y_train, y_val):
    print(s.shape)

(729, 82)
(487, 82)
(729,)
(487,)


In [13]:
print numpy.shape(y_train)

(729,)


In [14]:
print numpy.shape(X_train)

(729, 82)


In [24]:
def build_network(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None, 82),
                                     input_var=input_var)
    l_hid = lasagne.layers.DenseLayer(l_in, num_units=82, 
                                       nonlinearity=lasagne.nonlinearities.rectify,
                                       W=lasagne.init.GlorotUniform())
    l_hid2 = lasagne.layers.DenseLayer(l_hid, num_units=20, 
                                       nonlinearity=lasagne.nonlinearities.softmax,
                                       W=lasagne.init.GlorotUniform())
    l_out = lasagne.layers.DenseLayer(l_hid2, num_units=2, 
                                      nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_out

In [25]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    #print inputs
    #print targets
    if shuffle:
        indices = numpy.arange(len(inputs))
        numpy.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [26]:
input_var = T.matrix('inputs')
target_var = T.ivector('targets')

In [27]:
network = build_network(input_var)

In [28]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [29]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params,
                                            learning_rate=0.01,
                                            momentum=0.9)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(
                                                    test_prediction,
                                                    target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

In [30]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [31]:
for epoch in range(700):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        for batch in iterate_minibatches(X_train, y_train, 50, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, 50, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        # Then we print the results for this epoch:
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

  training loss:		0.696622
  validation loss:		0.686107
  validation accuracy:		64.67 %
  training loss:		0.686012
  validation loss:		0.690249
  validation accuracy:		54.00 %
  training loss:		0.690218
  validation loss:		0.691176
  validation accuracy:		52.67 %
  training loss:		0.690250
  validation loss:		0.690389
  validation accuracy:		53.33 %
  training loss:		0.689126
  validation loss:		0.688862
  validation accuracy:		54.67 %
  training loss:		0.688184
  validation loss:		0.687459
  validation accuracy:		54.89 %
  training loss:		0.686020
  validation loss:		0.683597
  validation accuracy:		56.00 %
  training loss:		0.678527
  validation loss:		0.683847
  validation accuracy:		56.67 %
  training loss:		0.682352
  validation loss:		0.693349
  validation accuracy:		52.67 %
  training loss:		0.692231
  validation loss:		0.692728
  validation accuracy:		50.44 %
  training loss:		0.691778
  validation loss:		0.691020
  validation accuracy:		52.67 %
  training loss:		0.691431
  val

  training loss:		0.687122
  validation loss:		0.687048
  validation accuracy:		53.33 %
  training loss:		0.686590
  validation loss:		0.685204
  validation accuracy:		54.89 %
  training loss:		0.683972
  validation loss:		0.683221
  validation accuracy:		56.22 %
  training loss:		0.679473
  validation loss:		0.662917
  validation accuracy:		66.67 %
  training loss:		0.683477
  validation loss:		0.689488
  validation accuracy:		52.67 %
  training loss:		0.688483
  validation loss:		0.687254
  validation accuracy:		53.33 %
  training loss:		0.687037
  validation loss:		0.685282
  validation accuracy:		54.67 %
  training loss:		0.685411
  validation loss:		0.684659
  validation accuracy:		54.67 %
  training loss:		0.683561
  validation loss:		0.682489
  validation accuracy:		55.56 %
  training loss:		0.681395
  validation loss:		0.678139
  validation accuracy:		56.44 %
  training loss:		0.672069
  validation loss:		0.653400
  validation accuracy:		65.11 %
  training loss:		0.699964
  val

  training loss:		0.632901
  validation loss:		0.685550
  validation accuracy:		60.00 %
  training loss:		0.623979
  validation loss:		0.595771
  validation accuracy:		67.56 %
  training loss:		0.596249
  validation loss:		0.596186
  validation accuracy:		69.78 %
  training loss:		0.583112
  validation loss:		0.618165
  validation accuracy:		68.67 %
  training loss:		0.591912
  validation loss:		0.640300
  validation accuracy:		67.11 %
  training loss:		0.601127
  validation loss:		0.573122
  validation accuracy:		72.89 %
  training loss:		0.578903
  validation loss:		0.592139
  validation accuracy:		70.22 %
  training loss:		0.594380
  validation loss:		0.589216
  validation accuracy:		69.33 %
  training loss:		0.582059
  validation loss:		0.557398
  validation accuracy:		74.22 %
  training loss:		0.607935
  validation loss:		0.605998
  validation accuracy:		67.56 %
  training loss:		0.589213
  validation loss:		0.556475
  validation accuracy:		74.44 %
  training loss:		0.574459
  val

  training loss:		0.547550
  validation loss:		0.522856
  validation accuracy:		75.78 %
  training loss:		0.556630
  validation loss:		0.525690
  validation accuracy:		77.33 %
  training loss:		0.548407
  validation loss:		0.513877
  validation accuracy:		76.89 %
  training loss:		0.535623
  validation loss:		0.531455
  validation accuracy:		75.78 %
  training loss:		0.555301
  validation loss:		0.514369
  validation accuracy:		76.89 %
  training loss:		0.541114
  validation loss:		0.516758
  validation accuracy:		76.67 %
  training loss:		0.557047
  validation loss:		0.529069
  validation accuracy:		75.78 %
  training loss:		0.563389
  validation loss:		0.696289
  validation accuracy:		60.67 %
  training loss:		0.654429
  validation loss:		0.526758
  validation accuracy:		76.44 %
  training loss:		0.553046
  validation loss:		0.557979
  validation accuracy:		72.67 %
  training loss:		0.576750
  validation loss:		0.558356
  validation accuracy:		72.67 %
  training loss:		0.556385
  val

  training loss:		0.524321
  validation loss:		0.533442
  validation accuracy:		76.44 %
  training loss:		0.530688
  validation loss:		0.503363
  validation accuracy:		76.67 %
  training loss:		0.570787
  validation loss:		0.513357
  validation accuracy:		76.67 %
  training loss:		0.543077
  validation loss:		0.517219
  validation accuracy:		76.67 %
  training loss:		0.521522
  validation loss:		0.535282
  validation accuracy:		75.33 %
  training loss:		0.549328
  validation loss:		0.516554
  validation accuracy:		75.11 %
  training loss:		0.523476
  validation loss:		0.504589
  validation accuracy:		77.11 %
  training loss:		0.530255
  validation loss:		0.500387
  validation accuracy:		77.56 %
  training loss:		0.524732
  validation loss:		0.514419
  validation accuracy:		76.89 %
  training loss:		0.526538
  validation loss:		0.506331
  validation accuracy:		76.00 %
  training loss:		0.520534
  validation loss:		0.507495
  validation accuracy:		77.11 %
  training loss:		0.526895
  val

  training loss:		0.555449
  validation loss:		0.495613
  validation accuracy:		78.44 %
  training loss:		0.536973
  validation loss:		0.489043
  validation accuracy:		77.78 %
  training loss:		0.510346
  validation loss:		0.489218
  validation accuracy:		77.56 %
  training loss:		0.523276
  validation loss:		0.500457
  validation accuracy:		77.56 %
  training loss:		0.535364
  validation loss:		0.558488
  validation accuracy:		74.44 %
  training loss:		0.520936
  validation loss:		0.610419
  validation accuracy:		68.67 %
  training loss:		0.549490
  validation loss:		0.648047
  validation accuracy:		70.89 %
  training loss:		0.538683
  validation loss:		0.497437
  validation accuracy:		76.67 %
  training loss:		0.535766
  validation loss:		0.598650
  validation accuracy:		72.22 %
  training loss:		0.512986
  validation loss:		0.515045
  validation accuracy:		76.67 %
  training loss:		0.514462
  validation loss:		0.500886
  validation accuracy:		78.22 %
  training loss:		0.548578
  val

  training loss:		0.522956
  validation loss:		0.493900
  validation accuracy:		77.78 %
  training loss:		0.515198
  validation loss:		0.487262
  validation accuracy:		79.33 %
  training loss:		0.514190
  validation loss:		0.524488
  validation accuracy:		75.11 %
  training loss:		0.518611
  validation loss:		0.486144
  validation accuracy:		79.56 %
  training loss:		0.645371
  validation loss:		0.557180
  validation accuracy:		74.89 %
  training loss:		0.532421
  validation loss:		0.505902
  validation accuracy:		78.67 %
  training loss:		0.521922
  validation loss:		0.503839
  validation accuracy:		78.67 %
  training loss:		0.516704
  validation loss:		0.488830
  validation accuracy:		79.56 %
  training loss:		0.521953
  validation loss:		0.534698
  validation accuracy:		74.00 %
  training loss:		0.517811
  validation loss:		0.488801
  validation accuracy:		77.56 %
  training loss:		0.582947
  validation loss:		0.562867
  validation accuracy:		71.33 %
  training loss:		0.534428
  val

  training loss:		0.498279
  validation loss:		0.471743
  validation accuracy:		79.33 %


In [33]:
numpy.savez('model_1.npz', *lasagne.layers.get_all_param_values(network))

In [34]:
def iterate_minibatches_2(inputs, batchsize):
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt]
        
        
# use trained network for predictions
test_prediction = lasagne.layers.get_output(network, deterministic=True)
predict_fn = theano.function([input_var], T.argmax(test_prediction, axis=1))

test_data = good
sum = 0
for batch in iterate_minibatches_2(inputs=test_data,batchsize=1):
        inputs = batch
        result=predict_fn(inputs)
        sum += result[0]
print len(good)
print sum

test_data = bad
sum = 0
for batch in iterate_minibatches_2(inputs=test_data,batchsize=1):
        inputs = batch
        result=predict_fn(inputs)
        sum += result[0]
print len(bad)
print len(bad)-sum
494+399

608
432
608
516


893